In [70]:
import pandas as pd
pd.options.display.max_columns = None

import numpy as np
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler

import os
from dotenv import load_dotenv
import boto3
import io
import pymysql
import json

DB Connection

In [11]:
# from sshtunnel import SSHTunnelForwarder

# class SSHMySQLConnector:
#     def __init__(self):
#         self.ssh_host = None
#         self.ssh_username = None
#         self.ssh_password = None
#         self.db_username = None
#         self.db_password = None
#         self.db_name = None
#         self.tunnel = None
#         self.connection = None

#     def load_config_from_json(self, json_path):
#         try:
#             with open(json_path, 'r', encoding='utf-8') as f:
#                 config = json.load(f)
#                 self.ssh_host = config['ssh_host']
#                 self.ssh_username = config['ssh_username']
#                 self.ssh_password = config['ssh_password']
#                 self.db_username = config['db_username']
#                 self.db_password = config['db_password']
#                 self.db_name = config['db_name']
#         except Exception as e:
#             print("설정 JSON 로딩 실패:", e)

#     def connect(self):
#         try:
#             self.tunnel = SSHTunnelForwarder(
#                 (self.ssh_host, 22),
#                 ssh_username=self.ssh_username,
#                 ssh_password=self.ssh_password,
#                 remote_bind_address=('127.0.0.1', 3306),
#             )
#             self.tunnel.start()
            
#             self.connection = pymysql.connect(
#                 host='127.0.0.1',
#                 port=self.tunnel.local_bind_port,
#                 user=self.db_username,
#                 password=self.db_password,
#                 db=self.db_name,
#                 cursorclass=pymysql.cursors.DictCursor  # 이 줄 추가
#             )
#             print("DB 접속 성공")
#         except Exception as e:
#             print("SSH 또는 DB 연결 실패:", e)

#     def insert_query_with_lookup(self, table_name, data_list):
#         try:
#             with self.connection.cursor() as cursor:
#                 for data in data_list:
#                     # 1. op_member에서 uid, user_id 조회
#                     cursor.execute("""
#                         SELECT uid, user_id, add1_connected FROM op_member
#                         WHERE add1 = %s
#                         LIMIT 1
#                     """, (data['acnt_nm'],))
#                     result = cursor.fetchone()
                    
#                     if result:
#                         data['member_uid'] = result['uid']
#                         data['user_id'] = result['user_id']
#                         data['is_connected'] = result['add1_connected']
#                         # 향후에 ig_user_id가 추가가 된다면, 해당 부분도 확인해서 추가할 수 있게
#                         # data['ig_user_id'] = result['ig_user_id']
#                     else:
#                         data['member_uid'] = 0
#                         data['user_id'] = 'None'
#                         data['is_connected'] = 'n'
#                         # data['ig_user_id'] = 'None'
              

#                     # 2. INSERT 쿼리 구성 및 실행
#                     columns = ', '.join(data.keys())
#                     placeholders = ', '.join([f"%({k})s" for k in data.keys()])
#                     insert_sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
#                     print(insert_sql)
#                     cursor.execute(insert_sql, data)

#                     print(f"inserted acnt_id: {data.get('acnt_id', 'N/A')}")

#             self.connection.commit()
#         except Exception as e:
#             self.connection.rollback()
#             print("INSERT 실패:", e)

#     def get_all_infos():
        
#         return 
    
#     def close(self):
#         if self.connection:
#             self.connection.close()
#         if self.tunnel:
#             self.tunnel.stop()

In [12]:
# ssh = SSHMySQLConnector()
# ssh.load_config_from_json('C:/Users/ehddl/Desktop/업무/code/config/ssh_db_config.json')
# ssh.connect()

# data_list = [
#     {
#         'activity_score': 0.18618008163528707,
#         'trend_score': 1.2384506112322682,
#         'follower_total_engagement': 0.9006925508419993,
#         'follower_retention_rate': 4.84593837535013,
#         'avg_post_efficiency': 1.4776093002849746,
#         'acnt_id': '17841400070132367',
#         'acnt_nm': 'gnuoyeatt',
#         'influencer_scale_type': 'nano'
#     }
# ]

# ssh.insert_query_with_lookup('op_mem_seller_score', data_list)


In [72]:
from sshtunnel import SSHTunnelForwarder

class SSHMySQLConnector:
    def __init__(self):
        self.ssh_host = None
        self.ssh_username = None
        self.ssh_password = None
        self.db_username = None
        self.db_password = None
        self.db_name = None
        self.tunnel = None
        self.connection = None

    def load_config_from_json(self, json_path):
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                config = json.load(f)
                self.ssh_host = config['ssh_host']
                self.ssh_username = config['ssh_username']
                self.ssh_password = config['ssh_password']
                self.db_username = config['db_username']
                self.db_password = config['db_password']
                self.db_name = config['db_name']
        except Exception as e:
            print("설정 JSON 로딩 실패:", e)

    def connect(self, insert=False):
        try:
            self.tunnel = SSHTunnelForwarder(
                (self.ssh_host, 22),
                ssh_username=self.ssh_username,
                ssh_password=self.ssh_password,
                remote_bind_address=('127.0.0.1', 3306),
            )
            self.tunnel.start()
            # insert 여부에 따라 cursorclass 설정
            connect_kwargs = {
                'host': '127.0.0.1',
                'port': self.tunnel.local_bind_port,
                'user': self.db_username,
                'password': self.db_password,
                'db': self.db_name,
            }
            if insert:
                connect_kwargs['cursorclass'] = pymysql.cursors.DictCursor
            self.connection = pymysql.connect(**connect_kwargs)
            print("DB 접속 성공")
        except Exception as e:
            print("SSH 또는 DB 연결 실패:", e)

    def execute_query(self, query):
        # 쿼리 실행 후 데이터를 DataFrame으로 반환
        return pd.read_sql_query(query, self.connection)

    def insert_query_with_lookup(self, table_name, data_list):
        try:
            with self.connection.cursor() as cursor:
                for data in data_list:
                    # 1. op_member에서 uid, user_id 조회
                    cursor.execute("""
                        SELECT uid, user_id, add1_connected FROM op_member
                        WHERE add1 = %s
                        LIMIT 1
                    """, (data['acnt_nm'],))
                    result = cursor.fetchone()
                    
                    if result:
                        data['member_uid'] = result['uid']
                        data['user_id'] = result['user_id']
                        data['is_connected'] = result['add1_connected']
                        # 향후에 ig_user_id가 추가가 된다면, 해당 부분도 확인해서 추가할 수 있게
                        # data['ig_user_id'] = result['ig_user_id']
                    else:
                        data['member_uid'] = 0
                        data['user_id'] = 'None'
                        data['is_connected'] = 'n'
                        # data['ig_user_id'] = 'None'
              

                    # 2. INSERT 쿼리 구성 및 실행
                    columns = ', '.join(data.keys())
                    placeholders = ', '.join([f"%({k})s" for k in data.keys()])
                    insert_sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
                    print(insert_sql)
                    cursor.execute(insert_sql, data)

                    print(f"inserted acnt_id: {data.get('acnt_id', 'N/A')}")

            self.connection.commit()
        except Exception as e:
            self.connection.rollback()
            print("INSERT 실패:", e)
    
    def close(self):
        if self.connection:
            self.connection.close()
        if self.tunnel:
            self.tunnel.stop()

def sendQuery(query):
        ssh = SSHMySQLConnector()
        ssh.load_config_from_json('C:/Users/ehddl/Desktop/업무/code/config/ssh_db_config.json')
        ssh.connect()
        results = ssh.execute_query(query)
        # print(results)
        # print(results.head())
        ssh.close()

        return results
    
def get_all_infos(): 

    query_sales_info = """
        SELECT o.uid, o.add1, s.*
        FROM op_mem_seller_statistics s
        JOIN (
            SELECT member_uid, MAX(regdate) AS max_regdate
            FROM op_mem_seller_statistics
            GROUP BY member_uid
        ) latest ON s.member_uid = latest.member_uid AND s.regdate = latest.max_regdate
        JOIN op_member o ON o.uid = s.member_uid
        JOIN S3_RECENT_USER_INFO_MTR u ON o.add1 = u.acnt_nm
        ORDER BY s.uid DESC
    """
    sales_info = sendQuery(query_sales_info)

    query_seller_info = """
        SELECT
        o.user_id, o.ig_user_id, o.add1, s.interestcategory
        FROM op_member o
        left join op_mem_seller s on o.user_id=s.user_id
        where (o.ig_user_id != '' and o.ig_user_id is not null) or (o.add1 != '' and o.add1 is not null)
    """
    seller_info = sendQuery(query_seller_info)

    return sales_info, seller_info


In [73]:
ssh = SSHMySQLConnector()
ssh.load_config_from_json('C:/Users/ehddl/Desktop/업무/code/config/ssh_db_config.json')
ssh.connect(True)

DB 접속 성공


In [74]:
sales_info, seller_info = get_all_infos()

DB 접속 성공
DB 접속 성공


In [16]:
# data_list = [
#     {
#         'acnt_id': '17841400070132367',
#         'acnt_nm': 'gnuoyeatt',
#         'influencer_scale_type': 'nano'
#     }
# ]

# ssh = SSHMySQLConnector()
# ssh.load_config_from_json('C:/Users/ehddl/Desktop/업무/code/config/ssh_db_config.json')
# ssh.connect(True) # insert 할 때는 True 값을 넣어줘야 함
# ssh.insert_query_with_lookup('op_mem_seller_score', data_list)

Data Loading

In [77]:
def load_s3_instagram_data():

    load_dotenv()
    aws_access_key = os.getenv("aws_accessKey")
    aws_secret_key = os.getenv("aws_secretKey")
    region_name='ap-northeast-2'

    # S3 클라이언트 생성
    client = boto3.client('s3', aws_access_key_id=aws_access_key, aws_secret_access_key=aws_secret_key, region_name=region_name)

    today = datetime.now()
    year, week, _ = today.isocalendar()

    bucket_name = 'flexmatch-data'

    table_list = [
        'CONN_v2_RECENT_USER_INFO_MTR',
        'CONN_v2_TIME_SERIES_PROFILE_INFO',
        'CONN_v2_BY_USER_ID_MEDIA_DTL_INFO',
        'CONN_v2_BY_DATE_MEDIA_AGG_INFO',
        'CONN_v2_PROFILE_INSIGHT_DTL',
        'CONN_v2_MEDIA_INSIGHT_CUM',
        'CONN_v2_PROFILE_INSIGHT_FOLLOWTYPE']
    
    df_by_table_list = {}

    for table_name in table_list:
        prefix = f'instagram-data/tables/{table_name}/2025-07-16/' # 이번주가 없어서 week를 따로 x
        response = client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

        if 'Contents' in response:
            files = [content['Key'] for content in response['Contents'] if content['Key'].endswith('merged_data.parquet')]

            if not files:
                print(f"[Info] No merged_data.parquet found for table: {table_name}")
                continue

            # 각 파일 순회
            for file_key in files:
                try:
                    obj = client.get_object(Bucket=bucket_name, Key=file_key)
                    df = pd.read_parquet(io.BytesIO(obj['Body'].read()))
                    df_by_table_list[table_name] = df
                    print(f"[Success] Loaded {file_key} for table {table_name}")
                except Exception as e:
                    print(f"[Error] Failed to read {file_key} for table {table_name}: {e}")
        else:
            print(f"[Info] No files found under prefix: {prefix}")
        
    return df_by_table_list

df_by_table_list = load_s3_instagram_data()

# recent_user_info_mtr_2 = df_by_table_list['CONN_v2_RECENT_USER_INFO_MTR']
# time_series_profile_info_2 = df_by_table_list['CONN_v2_TIME_SERIES_PROFILE_INFO']
# by_user_id_media_dtl_info_2 = df_by_table_list['CONN_v2_BY_USER_ID_MEDIA_DTL_INFO']
# by_date_media_agg_info_2 = df_by_table_list['CONN_v2_BY_DATE_MEDIA_AGG_INFO']
# conn_profile_insight_2 = df_by_table_list['CONN_v2_PROFILE_INSIGHT_DTL']
# conn_media_insight_2 = df_by_table_list['CONN_v2_MEDIA_INSIGHT_CUM']
# conn_profile_insight_followtype_2 = df_by_table_list['CONN_v2_PROFILE_INSIGHT_FOLLOWTYPE']

recent_user_info_mtr = df_by_table_list['CONN_v2_RECENT_USER_INFO_MTR']
time_series_profile_info = df_by_table_list['CONN_v2_TIME_SERIES_PROFILE_INFO']
by_user_id_media_dtl_info = df_by_table_list['CONN_v2_BY_USER_ID_MEDIA_DTL_INFO']
by_date_media_agg_info = df_by_table_list['CONN_v2_BY_DATE_MEDIA_AGG_INFO']
conn_profile_insight = df_by_table_list['CONN_v2_PROFILE_INSIGHT_DTL']
conn_media_insight = df_by_table_list['CONN_v2_MEDIA_INSIGHT_CUM']
conn_profile_insight_followtype = df_by_table_list['CONN_v2_PROFILE_INSIGHT_FOLLOWTYPE']




[Success] Loaded instagram-data/tables/CONN_v2_RECENT_USER_INFO_MTR/2025-07-16/merged_data.parquet for table CONN_v2_RECENT_USER_INFO_MTR
[Success] Loaded instagram-data/tables/CONN_v2_TIME_SERIES_PROFILE_INFO/2025-07-16/merged_data.parquet for table CONN_v2_TIME_SERIES_PROFILE_INFO
[Success] Loaded instagram-data/tables/CONN_v2_BY_USER_ID_MEDIA_DTL_INFO/2025-07-16/merged_data.parquet for table CONN_v2_BY_USER_ID_MEDIA_DTL_INFO
[Success] Loaded instagram-data/tables/CONN_v2_BY_DATE_MEDIA_AGG_INFO/2025-07-16/merged_data.parquet for table CONN_v2_BY_DATE_MEDIA_AGG_INFO
[Success] Loaded instagram-data/tables/CONN_v2_PROFILE_INSIGHT_DTL/2025-07-16/merged_data.parquet for table CONN_v2_PROFILE_INSIGHT_DTL
[Success] Loaded instagram-data/tables/CONN_v2_MEDIA_INSIGHT_CUM/2025-07-16/merged_data.parquet for table CONN_v2_MEDIA_INSIGHT_CUM
[Success] Loaded instagram-data/tables/CONN_v2_PROFILE_INSIGHT_FOLLOWTYPE/2025-07-16/merged_data.parquet for table CONN_v2_PROFILE_INSIGHT_FOLLOWTYPE


In [75]:
def load_weekly_instagram_data(bucket_name, table_list, target_filename='merged_data.parquet'):
    # 환경 변수 로딩
    load_dotenv()
    aws_access_key = os.getenv("aws_accessKey")
    aws_secret_key = os.getenv("aws_secretKey")

    client = boto3.client(
        's3',
        aws_access_key_id=aws_access_key,
        aws_secret_access_key=aws_secret_key,
        region_name='ap-northeast-2'
    )

    today = datetime.now()
    yesterday = (today - timedelta(days=1))

    today_date = datetime.now().strftime('%Y-%m-%d')
    yesterday_date = yesterday.strftime('%Y-%m-%d')

    # 결과 저장용 딕셔너리 초기화
    merged_data_by_table = {table_name: {} for table_name in table_list}

    # 주차별로 데이터 로딩
    recent_dates = [yesterday_date, today_date]
    recent_data_by_table = {}

    for table_name in table_list:
        recent_data_by_table[table_name] = {}

        for date_str in recent_dates:
            prefix = f'instagram-data/tables/{table_name}/{date_str}/'
            response = client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

            if 'Contents' not in response:
                print(f"[Info] No files found under prefix: {prefix}")
                continue

            target_files = [
                content['Key']
                for content in response['Contents']
                if content['Key'].endswith(target_filename)
            ]

            if not target_files:
                print(f"[Info] No {target_filename} found for {table_name} on date={date_str}")
                continue

            for file_key in target_files:
                try:
                    obj = client.get_object(Bucket=bucket_name, Key=file_key)
                    df = pd.read_parquet(io.BytesIO(obj['Body'].read()))
                    recent_data_by_table[table_name][date_str] = df
                    print(f"[Success] Loaded {file_key} for table {table_name}, date={date_str}")
                except Exception as e:
                    print(f"[Error] Failed to read {file_key} for {table_name}, date={date_str}: {e}")

    final_data = {}
    for table_name, date_data in recent_data_by_table.items():
        if today_date in date_data and yesterday_date in date_data:
            final_data[table_name] = {
                'yesterday': date_data[yesterday_date],
                'today': date_data[today_date]
            }
        else:
            print(f"[Warning] Missing yesterday or today data for table {table_name}")

    return final_data


In [19]:
bucket_name = 'flexmatch-data'
table_list = [
    'CONN_v2_RECENT_USER_INFO_MTR',
    'CONN_v2_TIME_SERIES_PROFILE_INFO',
    'CONN_v2_BY_USER_ID_MEDIA_DTL_INFO',
    'CONN_v2_BY_DATE_MEDIA_AGG_INFO',
    'CONN_v2_PROFILE_INSIGHT_DTL',
    'CONN_v2_MEDIA_INSIGHT_CUM',
    'CONN_v2_PROFILE_INSIGHT_FOLLOWTYPE'
]

merged_data_by_table = load_weekly_instagram_data(bucket_name, table_list)


[Success] Loaded instagram-data/tables/CONN_v2_RECENT_USER_INFO_MTR/2025-07-17/merged_data.parquet for table CONN_v2_RECENT_USER_INFO_MTR, date=2025-07-17
[Success] Loaded instagram-data/tables/CONN_v2_RECENT_USER_INFO_MTR/2025-07-18/merged_data.parquet for table CONN_v2_RECENT_USER_INFO_MTR, date=2025-07-18
[Success] Loaded instagram-data/tables/CONN_v2_TIME_SERIES_PROFILE_INFO/2025-07-17/merged_data.parquet for table CONN_v2_TIME_SERIES_PROFILE_INFO, date=2025-07-17
[Success] Loaded instagram-data/tables/CONN_v2_TIME_SERIES_PROFILE_INFO/2025-07-18/merged_data.parquet for table CONN_v2_TIME_SERIES_PROFILE_INFO, date=2025-07-18
[Success] Loaded instagram-data/tables/CONN_v2_BY_USER_ID_MEDIA_DTL_INFO/2025-07-17/merged_data.parquet for table CONN_v2_BY_USER_ID_MEDIA_DTL_INFO, date=2025-07-17
[Success] Loaded instagram-data/tables/CONN_v2_BY_USER_ID_MEDIA_DTL_INFO/2025-07-18/merged_data.parquet for table CONN_v2_BY_USER_ID_MEDIA_DTL_INFO, date=2025-07-18
[Success] Loaded instagram-data/ta

In [20]:
recent_user_info_mtr = merged_data_by_table['CONN_v2_RECENT_USER_INFO_MTR']['yesterday']
time_series_profile_info = merged_data_by_table['CONN_v2_TIME_SERIES_PROFILE_INFO']['yesterday']
by_user_id_media_dtl_info = merged_data_by_table['CONN_v2_BY_USER_ID_MEDIA_DTL_INFO']['yesterday']
by_date_media_agg_info = merged_data_by_table['CONN_v2_BY_DATE_MEDIA_AGG_INFO']['yesterday']

recent_user_info_mtr_2 = merged_data_by_table['CONN_v2_RECENT_USER_INFO_MTR']['today']
time_series_profile_info_2 = merged_data_by_table['CONN_v2_TIME_SERIES_PROFILE_INFO']['today']
by_user_id_media_dtl_info_2 = merged_data_by_table['CONN_v2_BY_USER_ID_MEDIA_DTL_INFO']['today']
by_date_media_agg_info_2 = merged_data_by_table['CONN_v2_BY_DATE_MEDIA_AGG_INFO']['today']

conn_profile_insight = merged_data_by_table['CONN_v2_PROFILE_INSIGHT_DTL']['yesterday']
conn_profile_insight_2 = merged_data_by_table['CONN_v2_PROFILE_INSIGHT_DTL']['today']

conn_media_insight = merged_data_by_table['CONN_v2_MEDIA_INSIGHT_CUM']['yesterday']
conn_media_insight_2 = merged_data_by_table['CONN_v2_MEDIA_INSIGHT_CUM']['today']

conn_profile_insight_followtype = merged_data_by_table['CONN_v2_PROFILE_INSIGHT_FOLLOWTYPE']['yesterday']
conn_profile_insight_followtype_2 = merged_data_by_table['CONN_v2_PROFILE_INSIGHT_FOLLOWTYPE']['today']

In [69]:
conn_media_insight_2[conn_media_insight_2['acnt_id']=='17841450980480576']

,acnt_id,acnt_nm,media_id,acnt_conn_yn,media_type_nm,reels_feed_type_nm,reg_dt,media_cn,like_cnt,share_cnt,views_cnt,save_cnt,cmnt_cnt,reach_cnt,follow_cnt,profile_visit_cnt,profile_activ_cnt,contents_intac_cnt,reels_view_total_time,reels_view_avg_time,base_ymd


basic preprocessing

In [21]:
# recent_user_info_mtr = recent_user_info_mtr.drop_duplicates(subset=['acnt_id', 'acnt_nm'])
# time_series_profile_info = time_series_profile_info.drop_duplicates(subset=['base_ymd', 'acnt_id'])
# by_user_id_media_dtl_info = by_user_id_media_dtl_info.drop_duplicates(subset=['acnt_id', 'media_id', 'media_cn'])
# by_date_media_agg_info = by_date_media_agg_info.drop_duplicates(subset=['base_ymd', 'media_id'])

# recent_user_info_mtr_2 = recent_user_info_mtr_2.drop_duplicates(subset=['acnt_id', 'acnt_nm'])
# time_series_profile_info_2 = time_series_profile_info_2.drop_duplicates(subset=['base_ymd', 'acnt_id'])
# by_user_id_media_dtl_info_2 = by_user_id_media_dtl_info_2.drop_duplicates(subset=['acnt_id', 'media_id', 'media_cn'])
# by_date_media_agg_info_2 = by_date_media_agg_info_2.drop_duplicates(subset=['base_ymd', 'media_id'])

# conn_profile_insight = conn_profile_insight.drop_duplicates(subset=['acnt_id', 'acnt_nm'])
# conn_media_insight = conn_media_insight.drop_duplicates(subset=['acnt_id', 'acnt_nm', 'media_id'])

# conn_profile_insight_2 = conn_profile_insight_2.drop_duplicates(subset=['acnt_id', 'acnt_nm'])
# conn_media_insight_2 = conn_media_insight_2.drop_duplicates(subset=['acnt_id', 'acnt_nm', 'media_id'])


가장 최근 데이터로 isin 해줘야 할 것 같음. 그래야 전, 후를 비교할 수 있기 때문임

In [78]:
# 각 acnt_id가 있는 전부 있는 경우만 가능, isin으로 제거
# 근데 전 주차에는 없는 유저가 생길수도 있음
# media 같은 경우에도 최근 100개만 일단 가져오고 있기 때문에 일주일 단위여도 없을수도 있음

# 이렇게 해야하는 이유는 timeseries 변수를 계산하려면 무조건 두 주차에 같은 아이디가 있어야 함
# 또한, user info랑 media info 같이 써야하는 경우에는 어짜피 한번 거르고 있을 듯
# -> 따라서, 굳이 필수적으로 하지는 않아도 될듯함

# unique_user = recent_user_info_mtr['acnt_id'].unique()
c_unique_user = recent_user_info_mtr_2[recent_user_info_mtr_2['acnt_conn_yn']=='Y']['acnt_id'].to_list()
c_recent_user_info_mtr_2 = recent_user_info_mtr_2[recent_user_info_mtr_2['acnt_id'].isin(c_unique_user)]

c_time_series_profile_info = time_series_profile_info[time_series_profile_info['acnt_id'].isin(c_unique_user)]
c_time_series_profile_info_2 = time_series_profile_info_2[time_series_profile_info_2['acnt_id'].isin(c_unique_user)]

# by_user_id_media_dtl_info = by_user_id_media_dtl_info[by_user_id_media_dtl_info['acnt_id'].isin(c_unique_user)]
c_by_user_id_media_dtl_info_2 = by_user_id_media_dtl_info_2[by_user_id_media_dtl_info_2['acnt_id'].isin(c_unique_user)]

c_conn_profile_insight = conn_profile_insight[conn_profile_insight['acnt_id'].isin(c_unique_user)]
c_conn_profile_insight_2 = conn_profile_insight_2[conn_profile_insight_2['acnt_id'].isin(c_unique_user)]

c_conn_profile_insight_followtype = conn_profile_insight_followtype[conn_profile_insight_followtype['acnt_id'].isin(c_unique_user)]
c_conn_profile_insight_followtype_2 = conn_profile_insight_followtype_2[conn_profile_insight_followtype_2['acnt_id'].isin(c_unique_user)]

# connected_user 같은 경우에는 conn_media_insight 안에 있는 게 media_agg랑 동일하기 때문에 해당 부분을 쓰면 당장은 문제가 없음
unique_media = c_by_user_id_media_dtl_info_2['media_id'].unique()
# c_by_date_media_agg_info_2 = by_date_media_agg_info_2[by_date_media_agg_info_2['media_id'].isin(unique_media)]
c_conn_media_insight_2 = conn_media_insight_2[conn_media_insight_2['media_id'].isin(unique_media)]

In [55]:
# pd.options.display.max_columns = None
# c_conn_media_insight_2[(c_conn_media_insight_2['acnt_id']=='17841453615191128')&(c_conn_media_insight_2['views_cnt']==0)].sort_values(by=['views_cnt'], ascending=True)

In [81]:
def influencer_scale_type(row):
    count = row['follower_cnt']
    if count < 1000:
        return 'nano'
    elif 1000 <= count <= 10000:
        return 'micro'
    elif 10000 < count <= 100000:
        return 'mid'
    elif 100000 < count <= 500000:
        return 'macro'
    else:
        return 'mega'

# recent_user_info_mtr.loc[:, 'influencer_scale_type'] = recent_user_info_mtr.apply(influencer_scale_type, axis=1)
c_recent_user_info_mtr_2.loc[:,'influencer_scale_type'] = c_recent_user_info_mtr_2.apply(influencer_scale_type, axis=1)


create merged file

In [82]:
def create_merged_df(user_info_df, timeseries_df, timeseries_df_2, media_info_df, media_insight_df, user_followtype_df, user_followtype_df_2): # media_agg, profile_insight X
    # merge 시에 같은 이름의 열이 두개여서 error 발생하기 때문에 insight에서는 삭제
    media_insight = media_insight_df.drop(['acnt_id'], axis=1)
    media_engagement_merged_df = pd.merge(media_info_df, media_insight, on='media_id', how='outer')
    # print(len(media_engagement_merged_df['acnt_id'].unique()))

    ### 방법 1
    # 단 한개의 게시물이라도 like가 비공개인 influencer 제거 & media_cnt_가 0인 사람도 제외
    # by_user_na_like_count = media_engagement_merged_df[media_engagement_merged_df['like_cnt'].isna()].groupby(['acnt_id'])['media_id'].count()
    # na_like_user = by_user_na_like_count[by_user_na_like_count > 0].index
    # no_media_user = user_info_df[user_info_df['media_cnt'] == 0]['acnt_id'].to_list()
    # except_user = list(na_like_user) + no_media_user
    # media_engagement_merged_df = media_engagement_merged_df[~media_engagement_merged_df['acnt_id'].isin(except_user)].reset_index()

    ### 방법 2
    # 미디어가 한 개도 없는 유저 제거 & 게시물의 like가 비공개인 경우에는 그걸 제외한 게시물의 좋아요 평균으로 채워넣기
    # 근데 여기서 like만 비공인건지 나머지 값들도 비공이 되는건지 확인을 하긴 해야함
    # mean 값 자체가 nan인 사람들도 제외
    # 근데 여기서 프로페셔널 전환은 이미 되어있지만, views라는 지표 자체가 나중에 나와서 전부 0으로 찍히는 경우가 있음. -> 어떻게 해결?
    no_media_user = user_info_df[user_info_df['media_cnt'] == 0]['acnt_id'].to_list()
    media_engagement_merged_df = media_engagement_merged_df[~media_engagement_merged_df['acnt_id'].isin(no_media_user)].reset_index()

    media_engagement_merged_groupby_df = media_engagement_merged_df.groupby('acnt_id')[['like_cnt', 'cmnt_cnt', 'share_cnt', 'save_cnt', 'views_cnt', 'reach_cnt']].mean()
    media_engagement_merged_groupby_df = np.ceil(media_engagement_merged_groupby_df)
    fillna_user = media_engagement_merged_groupby_df[media_engagement_merged_groupby_df['like_cnt'] > 1].index

    media_engagement_merged_df = media_engagement_merged_df[media_engagement_merged_df['acnt_id'].isin(fillna_user)].reset_index()

    engagement_cols = ['like_cnt', 'cmnt_cnt', 'share_cnt', 'save_cnt', 'views_cnt', 'reach_cnt']
    for col in engagement_cols:
        media_engagement_merged_df[col] = media_engagement_merged_df.apply(
        lambda row: media_engagement_merged_groupby_df.at[row['acnt_id'], col] if pd.isna(row[col]) else row[col], axis=1)

    user_list = media_engagement_merged_df['acnt_id'].unique()
    media_list = media_engagement_merged_df['media_id'].unique()

    # merge하면서 제거된 리스트가 있기 때문에, 해당 부분 다시 삭제 후에 새로운 merge 파일 생성
    user_info = user_info_df[user_info_df['acnt_id'].isin(user_list)]
    timeseries = timeseries_df[timeseries_df['acnt_id'].isin(user_list)]
    timeseries_2 = timeseries_df_2[timeseries_df_2['acnt_id'].isin(user_list)]
    media_info = media_info_df[media_info_df['acnt_id'].isin(user_list)]
    
    user_followtype = user_followtype_df[user_followtype_df['acnt_id'].isin(user_list)]
    user_followtype_2 = user_followtype_df_2[user_followtype_df_2['acnt_id'].isin(user_list)]
    
    media_insight_info = media_insight[media_insight['media_id'].isin(media_list)]

    all_merged_df_a = pd.merge(user_info, timeseries, on='acnt_id')
    all_merged_df_b = pd.merge(all_merged_df_a, media_info, on='acnt_id')
    all_merged_df = pd.merge(all_merged_df_b, media_insight_info, on='media_id')
    
    media_engagement_profile_merged_df = pd.merge(media_engagement_merged_df, user_info_df, on='acnt_id')
    time_series_merged_df = pd.merge(timeseries, timeseries_df_2, on='acnt_id')

    return user_info, timeseries, timeseries_2, user_followtype, user_followtype_2, media_info, media_insight_info, all_merged_df, media_engagement_merged_df, media_engagement_profile_merged_df, time_series_merged_df

In [83]:
user_info, timeseries, timeseries_2, user_followtype, user_followtype_2, media_info, media_agg, all_merged_df, media_engagement_merged_df, media_engagement_profile_merged_df, time_series_merged_df = create_merged_df(
                                                                                                                        c_recent_user_info_mtr_2,
                                                                                                                        c_time_series_profile_info,
                                                                                                                        c_time_series_profile_info_2,
                                                                                                                        c_by_user_id_media_dtl_info_2,
                                                                                                                        c_conn_media_insight_2,
                                                                                                                        c_conn_profile_insight_followtype,
                                                                                                                        c_conn_profile_insight_followtype_2)

In [84]:
media_engagement_profile_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422 entries, 0 to 421
Data columns (total 44 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   level_0                 422 non-null    int64  
 1   index                   422 non-null    int64  
 2   acnt_id                 422 non-null    object 
 3   media_id                422 non-null    object 
 4   media_type_nm_x         422 non-null    object 
 5   reels_feed_type_nm_x    422 non-null    object 
 6   media_url_addr          422 non-null    object 
 7   media_unq_url_addr      422 non-null    object 
 8   tmnl_url_addr           116 non-null    object 
 9   reg_dt_x                422 non-null    object 
 10  media_cn_x              372 non-null    object 
 11  acnt_conn_yn_x          422 non-null    object 
 12  feed_share_yn           108 non-null    object 
 13  cmnt_actvtn_yn          422 non-null    object 
 14  acnt_nm_x               422 non-null    ob

In [85]:
media_engagement_profile_merged_df[media_engagement_profile_merged_df['acnt_id']=='17841450980480576']

,level_0,index,acnt_id,media_id,media_type_nm_x,reels_feed_type_nm_x,media_url_addr,media_unq_url_addr,tmnl_url_addr,reg_dt_x,media_cn_x,acnt_conn_yn_x,feed_share_yn,cmnt_actvtn_yn,acnt_nm_x,acnt_conn_yn_y,media_type_nm_y,reels_feed_type_nm_y,reg_dt_y,media_cn_y,like_cnt,share_cnt,views_cnt,save_cnt,cmnt_cnt,reach_cnt,follow_cnt_x,profile_visit_cnt,profile_activ_cnt,contents_intac_cnt,reels_view_total_time,reels_view_avg_time,base_ymd,acnt_nm_y,web_addr,acnt_sub_nm,intro_txt,profile_photo_url_addr,acnt_conn_yn,category_nm,follower_cnt,follow_cnt_y,media_cnt,influencer_scale_type
7,7,7,17841450980480576,17845632434762696,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-2.cdninstagram.com/v/t51...,https://www.instagram.com/p/CcFHLyjJym7/,None,2022-04-08T06:23:29+0000,#34\n🔵 🧊🧊🧊🧊🧊\n클라임 어스 첫 방문\n시설 훌륭하네\n남색 어렵다\n클라...,Y,None,true,__dongx2_,Y,CAROUSEL_ALBUM,FEED,2022-04-08T06:23:29+0000,#34\n🔵 🧊🧊🧊🧊🧊\n클라임 어스 첫 방문\n시설 훌륭하네\n남색 어렵다\n클라...,156.0,0.0,18.0,1.0,6.0,60.0,NaN,NaN,NaN,NaN,NaN,NaN,2025-07-17,__dongx2_,https://open.kakao.com/o/gM2Q9pef,동일,"There's no bad conditions, there's only weakne...",None,Y,베이비/키즈,722,699,116,nano
18,18,18,17841450980480576,17850258761802532,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/CgqtlLjJ9Z9/,None,2022-07-31T06:56:29+0000,#64회차 \n⚫️⚫️❌️🔴🔴🔴🔴🔴🔴\n손상원 클라이밍짐 강남점 오픈런!\n처음 만...,Y,None,true,__dongx2_,Y,CAROUSEL_ALBUM,FEED,2022-07-31T06:56:29+0000,#64회차 \n⚫️⚫️❌️🔴🔴🔴🔴🔴🔴\n손상원 클라이밍짐 강남점 오픈런!\n처음 만...,245.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,248.0,NaN,NaN,2025-07-17,__dongx2_,https://open.kakao.com/o/gM2Q9pef,동일,"There's no bad conditions, there's only weakne...",None,Y,베이비/키즈,722,699,116,nano
23,24,24,17841450980480576,17856317744695367,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-2.cdninstagram.com/v/t51...,https://www.instagram.com/p/CZoOOpFpPUP/,None,2022-02-06T08:04:17+0000,.\n🧊🧊🧊🧊🧊 좋다 좀 늘긴했어ㅎㅎ\n🟢 다이노라서 재밌다\n🔵 이거 진짜 깨고 ...,Y,None,true,__dongx2_,Y,CAROUSEL_ALBUM,FEED,2022-02-06T08:04:17+0000,.\n🧊🧊🧊🧊🧊 좋다 좀 늘긴했어ㅎㅎ\n🟢 다이노라서 재밌다\n🔵 이거 진짜 깨고 ...,156.0,0.0,18.0,1.0,6.0,60.0,NaN,NaN,NaN,NaN,NaN,NaN,2025-07-17,__dongx2_,https://open.kakao.com/o/gM2Q9pef,동일,"There's no bad conditions, there's only weakne...",None,Y,베이비/키즈,722,699,116,nano
25,26,26,17841450980480576,17857189607706657,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/CdiBqPup5KM/,None,2022-05-14T08:24:35+0000,#45회차\n🔵🔵🔵🔵🔵🔵\n웨어하우스짐 첫 방문\n깔끔하고 문제도 재밌고 사장님도 ...,Y,None,true,__dongx2_,Y,CAROUSEL_ALBUM,FEED,2022-05-14T08:24:35+0000,#45회차\n🔵🔵🔵🔵🔵🔵\n웨어하우스짐 첫 방문\n깔끔하고 문제도 재밌고 사장님도 ...,156.0,0.0,18.0,1.0,6.0,60.0,NaN,NaN,NaN,NaN,NaN,NaN,2025-07-17,__dongx2_,https://open.kakao.com/o/gM2Q9pef,동일,"There's no bad conditions, there's only weakne...",None,Y,베이비/키즈,722,699,116,nano
29,30,30,17841450980480576,17859119504684701,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-2.cdninstagram.com/v/t51...,https://www.instagram.com/p/Ca3xraepfP3/,None,2022-03-09T05:34:03+0000,.\n오랜만에 양재 더클라임\n🔵x2 다이노\n참교육만 엄청 당했다\n우연히 만난\...,Y,None,true,__dongx2_,Y,CAROUSEL_ALBUM,FEED,2022-03-09T05:34:03+0000,.\n오랜만에 양재 더클라임\n🔵x2 다이노\n참교육만 엄청 당했다\n우연히 만난\...,156.0,0.0,18.0,1.0,6.0,60.0,NaN,NaN,NaN,NaN,NaN,NaN,2025-07-17,__dongx2_,https://open.kakao.com/o/gM2Q9pef,동일,"There's no bad conditions, there's only weakne...",None,Y,베이비/키즈,722,699,116,nano
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,392,392,17841450980480576,18229115269104698,CAROUSEL_ALBUM,FEED,https://scontent-nrt1-1.cdninstagram.com/v/t51...,https://www.instagram.com/p/Cb4ZC0qJaBM/,None,2022-04-03T07:49:25+0000,#33\n🟣 물보라든 꿀보라든 보라!!\n❌️ 기세를 몰아 다른 보라 도전했지만 탑...,Y,None,true,__dongx2_,Y,CAROUSEL_ALBUM,FEED,2022-04-03T07:49:25+0000,#33\n🟣 물보라든 꿀보라든 보라!!\n❌️ 기세를 몰아 다른 보라 도전했지만 탑...,156.0,0.0,18.0,1.0,6.0,60.0,NaN,NaN,NaN,NaN,NaN,NaN,2025-07-17,__dongx2_,https://open.kakao.com/o/gM2Q9pef,동일,"There's no bad conditions, t

크리에이터 활동성

- 실제 게시물 업로드 날짜를 기준으로 계산할 수 있을 것 같음
- 최근 100개의 게시물의 평균 게시 간격을 계산

In [86]:
def calculate_activity_score(recent_media_dtl_df): # 두 개의 테이블 중 가장 최근
    media_dtl_copy = recent_media_dtl_df.copy()
    media_dtl_copy = media_dtl_copy.drop_duplicates(subset=['acnt_id', 'media_id', 'media_cn'])
    media_dtl_copy['reg_dt'] = pd.to_datetime(media_dtl_copy['reg_dt'])
    media_dtl_copy = media_dtl_copy.sort_values(['acnt_id', 'reg_dt'])

    # 게시물 간격 계산
    media_dtl_copy['prev_reg_dt'] = media_dtl_copy.groupby('acnt_id')['reg_dt'].shift(1)
    media_dtl_copy['gap_days'] = (media_dtl_copy['reg_dt'] - media_dtl_copy['prev_reg_dt']).dt.days

    # gap_days가 NaN인 첫 번째 포스트 제외 후 평균 간격 계산
    activity_df = media_dtl_copy.dropna(subset=['gap_days']).groupby('acnt_id')['gap_days'].mean().reset_index()
    activity_df.rename(columns={'gap_days': 'avg_upload_interval'}, inplace=True)

    # 활동성 점수 계산 (간격의 역수로 환산) -> 점수 정규화 (업로드 간격이 짧을수록 점수가 높아지도록 역수를 취해서 계산한 것)
    # 업로드 간격이 너무 짧은 유저의 경우 inf로 계산되는 것을 방지하기 위해서 scaling 진행
    activity_df['avg_upload_interval'] = activity_df['avg_upload_interval'].replace(0, 0.1)
    # activity_df['activity_score'] =  activity_df['avg_upload_interval'] / 100
    activity_df['activity_score'] =  (1 / activity_df['avg_upload_interval']) * 100
    
    return activity_df

In [87]:
activity_df = calculate_activity_score(media_info)

In [88]:
activity_df

,acnt_id,avg_upload_interval,activity_score
0,17841400360358101,120.000000,0.833333
1,17841400624415491,21.545455,4.641350
2,17841409045873013,292.000000,0.342466
3,17841449549923448,3.131313,31.935484
4,17841450980480576,12.070707,8.284519
5,17841453615191128,0.979798,102.061856


In [89]:
def check_inf(df):
    float_cols = df.select_dtypes(include=['float64', 'float32']).columns

    mask_inf = np.isinf(df[float_cols]).any(axis=1)
    mask_neginf = np.isneginf(df[float_cols]).any(axis=1)

    invalid_rows = df[mask_inf | mask_neginf]

    print(f"⚠️ inf / -inf 포함 행 개수: {len(invalid_rows)}개")
    display(invalid_rows)

In [31]:
check_inf(activity_df)

⚠️ inf / -inf 포함 행 개수: 0개


,acnt_id,avg_upload_interval,activity_score


트렌드지수 - 팔로워 순 증가율 (follower_growth_rate)

In [32]:
# time_series_profile_info_2['acnt_id'] = time_series_profile_info_2['acnt_id'].astype(object)

# influencer_list = time_series_profile_info['acnt_id'].unique()
# time_series_profile_info_2[time_series_profile_info_2['acnt_id'].isin(influencer_list)]
# time_series_merged_df = pd.merge(time_series_profile_info, time_series_profile_info_2, on='acnt_id')

# time_series_merged_df['trend_score'] = ((time_series_merged_df['follower_cnt_y'] - time_series_merged_df['follower_cnt_x']) / (time_series_merged_df['follower_cnt_x'])) * 100
# time_series_merged_df[['acnt_id', 'trend_score']]

In [90]:
def calculate_follower_growth_rate(time_series_df, recent_time_series_df):
    time_series_df.loc[:, 'acnt_id'] = time_series_df['acnt_id'].astype(object)
    recent_time_series_df.loc[:, 'acnt_id'] = recent_time_series_df['acnt_id'].astype(object)

    influencer_list = time_series_df['acnt_id'].unique()
    recent_time_series_df = recent_time_series_df[recent_time_series_df['acnt_id'].isin(influencer_list)]
    time_series_merged_df = pd.merge(time_series_df, recent_time_series_df, on='acnt_id')

    time_series_merged_df['follow_growth_rate'] = ((time_series_merged_df['follower_cnt_y'] - time_series_merged_df['follower_cnt_x']) / (time_series_merged_df['follower_cnt_x'])) * 100
    growth_rate_df = time_series_merged_df[['acnt_id', 'follow_growth_rate']]

    return growth_rate_df

In [91]:
growth_rate_df = calculate_follower_growth_rate(timeseries, timeseries_2)

In [92]:
growth_rate_df

,acnt_id,follow_growth_rate
0,17841450980480576,0.000000
1,17841409045873013,0.000000
2,17841400360358101,-1.587302
3,17841449549923448,-0.037843
4,17841453615191128,0.033757
5,17841400624415491,0.000000


##### 팔로워 참여도

media_id는 있는데 프로필에 미디어가 없어서 0개인 사람이 있음


In [36]:
# def calculate_follower_engagement(media_engagement_profile_merged_df):
#     media_engagement_profile_merged_df_copy = media_engagement_profile_merged_df[['acnt_id', 'media_id', 'follower_cnt', 'like_cnt', 'cmnt_cnt', 'share_cnt', 'save_cnt', 'views_cnt', 'reach_cnt', 'media_cnt']]
#     # media_id는 조회가 되지만 실제로 media_cnt는 없는 경우가 있음
#     # media_engagement_profile_merged_df_copy = media_engagement_profile_merged_df_copy[media_engagement_profile_merged_df_copy['media_cnt'] != 0]
    
#     engaged_df = media_engagement_profile_merged_df_copy.groupby(['acnt_id']).agg({
#         'like_cnt' : 'sum',
#         'cmnt_cnt' : 'sum',
#         'share_cnt' : 'sum',
#         'save_cnt' : 'sum',
#         'media_id' : 'count',
#         'follower_cnt' : 'first',
#     }).reset_index()

#     engaged_df['estimated_total_engagement'] = ((engaged_df['like_cnt'] + engaged_df['cmnt_cnt'] + engaged_df['share_cnt'] + engaged_df['save_cnt']) / ( engaged_df['media_id']*engaged_df['follower_cnt']))
#     engaged_df['follower_total_engagement'] = engaged_df['estimated_total_engagement'] * 100
    
#     follower_engagment_df = engaged_df

#     return follower_engagment_df

In [37]:
# follower_engagment_df = calculate_follower_engagement(media_engagement_profile_merged_df)
# follower_engagment_df

In [38]:
# check_inf(follower_engagment_df)

##### 팔로워 충성도

- 비즈니스 계정 연동 시 follower_type 데이터 로딩 -> 실제 follower cnt랑 안맞음 (timeseries랑 안맞음)

In [93]:
def calculate_follower_loyalty(time_series_merged_df):
    time_series_merged_df_copy = time_series_merged_df[['acnt_id', 'follower_cnt_x', 'follower_cnt_y']].copy()

    time_series_merged_df_copy.loc[:, 'follower_change'] = (time_series_merged_df_copy['follower_cnt_y'] - time_series_merged_df_copy['follower_cnt_x'])

    def estimate_new_follower(row):
        if row['follower_change'] < 0:
            return 0
        else:
            return row['follower_change']

    time_series_merged_df_copy.loc[:, 'new_follower'] = time_series_merged_df_copy.apply(estimate_new_follower, axis=1)
    time_series_merged_df_copy.loc[:, 'unfollowed'] = time_series_merged_df_copy['follower_cnt_x'] + time_series_merged_df_copy['new_follower'] - time_series_merged_df_copy['follower_cnt_y']
    time_series_merged_df_copy.loc[:, 'follower_retention_rate'] = ((time_series_merged_df_copy['follower_cnt_x'] - time_series_merged_df_copy['unfollowed']) / time_series_merged_df_copy['follower_cnt_x']) * 100
    time_series_merged_df_copy.loc[:, 'follower_retention_rate'] = time_series_merged_df_copy['follower_retention_rate'].round(2)

    follower_loyalty_df = time_series_merged_df_copy

    return follower_loyalty_df

In [94]:
follower_loyalty_df = calculate_follower_loyalty(time_series_merged_df)
follower_loyalty_df

,acnt_id,follower_cnt_x,follower_cnt_y,follower_change,new_follower,unfollowed,follower_retention_rate
0,17841450980480576,722,722,0,0,0,100.00
1,17841409045873013,104,104,0,0,0,100.00
2,17841400360358101,126,124,-2,0,2,98.41
3,17841449549923448,50208,50189,-19,0,19,99.96
4,17841453615191128,35548,35560,12,12,0,100.00
5,17841400624415491,439,439,0,0,0,100.00


In [41]:
check_inf(follower_loyalty_df)

⚠️ inf / -inf 포함 행 개수: 0개


,acnt_id,follower_cnt_x,follower_cnt_y,follower_change,new_follower,unfollowed,follower_retention_rate


콘텐츠 효율성 평가

- 콘텐츠 1개당 팔로워 대비 반응량(like+comments+share+save)
- media_agg & media_dtl & acnt_id가 필요 -> 전부 병합한 게 media_engagement_merged_df
- 단, 프로페셔널 전환은 이전에 했으나, views라는 지표가 나중에 나와서 0으로 찍히는 애들이 있음 -> 이런 부분은 전부 inf로 계산이 되는데 전부 제거하고 mean을 구하는 것으로 !


In [95]:
def calculate_post_efficiency_df(media_engagement_profile_merged_df):
    media_engagement_profile_merged_df_copy = media_engagement_profile_merged_df.copy()

    media_engagement_profile_merged_df_copy['post_efficiency'] = ((media_engagement_profile_merged_df_copy['like_cnt'] + media_engagement_profile_merged_df_copy['cmnt_cnt'] + media_engagement_profile_merged_df_copy['save_cnt'] + media_engagement_profile_merged_df_copy['share_cnt']) / media_engagement_profile_merged_df_copy['follower_cnt']) * 100
    post_efficiency_df = media_engagement_profile_merged_df_copy.groupby('acnt_id')['post_efficiency'].mean().reset_index()
    post_efficiency_df.rename(columns={'post_efficiency': 'avg_post_efficiency'}, inplace=True)

    return post_efficiency_df

In [96]:
post_efficiency_df = calculate_post_efficiency_df(media_engagement_profile_merged_df)
post_efficiency_df

,acnt_id,avg_post_efficiency
0,17841400360358101,15.367384
1,17841400624415491,5.466970
2,17841409045873013,11.538462
3,17841449549923448,1.118891
4,17841450980480576,22.426593
5,17841453615191128,4.002109


콘텐츠 인기도 평가

- 콘텐츠 1개당 조회수 대비 반응량(like+comments+share+save)
- media_agg & media_dtl & acnt_id가 필요 -> 전부 병합한 게 media_engagement_merged_df
- 단, 프로페셔널 전환은 이전에 했으나, views라는 지표가 나중에 나와서 0으로 찍히는 애들이 있음 -> 이런 부분은 전부 inf로 계산이 되는데 전부 제거하고 mean을 구하는 것으로 !

In [97]:
def calculate_post_popularity_df(media_engagement_profile_merged_df):
    media_engagement_profile_merged_df_copy = media_engagement_profile_merged_df.copy()

    media_engagement_profile_merged_df_copy['post_popularity'] = ((media_engagement_profile_merged_df_copy['like_cnt'] + media_engagement_profile_merged_df_copy['cmnt_cnt'] + media_engagement_profile_merged_df_copy['save_cnt'] + media_engagement_profile_merged_df_copy['share_cnt']) / media_engagement_profile_merged_df_copy['views_cnt']) * 100
    # check_inf(media_engagement_profile_merged_df_copy)
    media_engagement_profile_merged_df_copy.replace([np.inf, -np.inf], np.nan, inplace=True)
    media_engagement_profile_merged_df_copy.dropna(subset=['post_popularity'], inplace=True)

    post_popularity_df = media_engagement_profile_merged_df_copy.groupby('acnt_id')['post_popularity'].mean().reset_index()
    post_popularity_df.rename(columns={'post_popularity': 'avg_post_popularity'}, inplace=True)

    return post_popularity_df

In [98]:
post_popularity_df = calculate_post_popularity_df(media_engagement_profile_merged_df)
post_popularity_df

,acnt_id,avg_post_popularity
0,17841400360358101,11.015802
1,17841400624415491,6.492680
2,17841409045873013,1.948052
3,17841449549923448,4.153314
4,17841450980480576,841.965152
5,17841453615191128,3.387946


##### connected_influencer_flexmatch_score

In [46]:
# def connected_user_flexmatch_score(activity_df, growth_rate_df, follower_engagement_df, follower_loyalty_df, post_efficiency_df, post_popularity_df):
#     # 크리에이터 활동성
#     creator_activity_score = activity_df[['acnt_id', 'activity_score']]
#     # 트렌드지수 (팔로워 순변화량)
#     creator_follow_growth_rate = growth_rate_df[['acnt_id', 'follow_growth_rate']] # db 변수명 수정
#     # 팔로워 참여도
#     follower_engagement = follower_engagement_df[['acnt_id', 'follower_total_engagement']]
#     # 팔로워 충성도
#     follower_loyalty = follower_loyalty_df[['acnt_id', 'follower_retention_rate']]
#     # 콘텐츠 효율성
#     post_efficiency = post_efficiency_df[['acnt_id', 'avg_post_efficiency']]
#     # 콘텐츠 인기도
#     post_popularity = post_popularity_df[['acnt_id', 'avg_post_popularity']]


#     # data_list
#     df_list = [creator_activity_score, creator_follow_growth_rate, follower_engagement, follower_loyalty, post_efficiency, post_popularity]

#     from functools import reduce

#     flexmatch_score = reduce(lambda left, right: pd.merge(left, right, on='acnt_id', how='left'), df_list)
#     user_info_nm = user_info[['acnt_id', 'acnt_nm', 'influencer_scale_type']]
#     flexmatch_score = pd.merge(flexmatch_score, user_info_nm, on='acnt_id')
#     flexmatch_score = flexmatch_score[['acnt_id', 'acnt_nm', 'influencer_scale_type', 'activity_score', 'follow_growth_rate', 'follower_total_engagement', 'follower_retention_rate', 'avg_post_efficiency', 'avg_post_popularity']]

#     connected_flexmatch_score_table = flexmatch_score.copy()
#     connected_flexmatch_score_table.dropna(inplace=True)
    
#     return connected_flexmatch_score_table

In [99]:
def connected_user_flexmatch_score(activity_df, growth_rate_df, follower_loyalty_df, post_efficiency_df, post_popularity_df):
    # 크리에이터 활동성
    creator_activity_score = activity_df[['acnt_id', 'activity_score']]
    # 트렌드지수 (팔로워 순변화량)
    creator_follow_growth_rate = growth_rate_df[['acnt_id', 'follow_growth_rate']] # db 변수명 수정
    # 팔로워 참여도
    # follower_engagement = follower_engagement_df[['acnt_id', 'follower_total_engagement']]
    # 팔로워 충성도
    follower_loyalty = follower_loyalty_df[['acnt_id', 'follower_retention_rate']]
    # 콘텐츠 효율성
    post_efficiency = post_efficiency_df[['acnt_id', 'avg_post_efficiency']]
    # 콘텐츠 인기도
    post_popularity = post_popularity_df[['acnt_id', 'avg_post_popularity']]


    # data_list
    df_list = [creator_activity_score, creator_follow_growth_rate, follower_loyalty, post_efficiency, post_popularity]

    from functools import reduce

    flexmatch_score = reduce(lambda left, right: pd.merge(left, right, on='acnt_id', how='left'), df_list)
    user_info_nm = user_info[['acnt_id', 'acnt_nm', 'influencer_scale_type']]
    flexmatch_score = pd.merge(flexmatch_score, user_info_nm, on='acnt_id')
    flexmatch_score = flexmatch_score[['acnt_id', 'acnt_nm', 'influencer_scale_type', 'activity_score', 'follow_growth_rate', 'follower_retention_rate', 'avg_post_efficiency', 'avg_post_popularity']]

    connected_flexmatch_score_table = flexmatch_score.copy()
    connected_flexmatch_score_table.dropna(inplace=True)
    
    return connected_flexmatch_score_table

In [100]:
connected_flexmatch_score_table = connected_user_flexmatch_score(activity_df, growth_rate_df, follower_loyalty_df, post_efficiency_df, post_popularity_df)
connected_flexmatch_score_table

,acnt_id,acnt_nm,influencer_scale_type,activity_score,follow_growth_rate,follower_retention_rate,avg_post_efficiency,avg_post_popularity
0,17841400360358101,bro_bums,nano,0.833333,-1.587302,98.41,15.367384,11.015802
1,17841400624415491,itsme_ddonni,nano,4.641350,0.000000,100.00,5.466970,6.492680
2,17841409045873013,__cherie_ssom,nano,0.342466,0.000000,100.00,11.538462,1.948052
3,17841449549923448,heihwi,mid,31.935484,-0.037843,99.96,1.118891,4.153314
4,17841450980480576,__dongx2_,nano,8.284519,0.000000,100.00,22.426593,841.965152
5,17841453615191128,bong_camper83,mid,102.061856,0.033757,100.00,4.002109,3.387946


In [101]:
db_merged_data = pd.merge(seller_info, sales_info, on='add1')
len(db_merged_data)

# -> 여기서 지금 merged_data의 개수가 작게 나오는거는 seller_info의 add1이랑 sales_info의 add1이 일치하지 않아서 그런 것 같음.
# 해당 부분

279

In [102]:
conn_user = seller_info[(seller_info['ig_user_id'].notnull()) & (seller_info['ig_user_id'] != '')]
conn_user_interestcategory = conn_user[['ig_user_id', 'interestcategory']].rename({'ig_user_id' : 'acnt_id'}, axis=1)
connected_flexmatch_score_table = pd.merge(connected_flexmatch_score_table, conn_user_interestcategory, on='acnt_id')

In [103]:
connected_flexmatch_score_table

,acnt_id,acnt_nm,influencer_scale_type,activity_score,follow_growth_rate,follower_retention_rate,avg_post_efficiency,avg_post_popularity,interestcategory
0,17841400360358101,bro_bums,nano,0.833333,-1.587302,98.41,15.367384,11.015802,패션@뷰티
1,17841400624415491,itsme_ddonni,nano,4.641350,0.000000,100.00,5.466970,6.492680,패션@뷰티@홈/리빙@푸드@헬시@스포츠@서비스@IT
2,17841409045873013,__cherie_ssom,nano,0.342466,0.000000,100.00,11.538462,1.948052,패션
3,17841449549923448,heihwi,mid,31.935484,-0.037843,99.96,1.118891,4.153314,패션
4,17841450980480576,__dongx2_,nano,8.284519,0.000000,100.00,22.426593,841.965152,베이비/키즈
5,17841453615191128,bong_camper83,mid,102.061856,0.033757,100.00,4.002109,3.387946,홈/리빙@푸드@베이비/키즈


In [96]:
# connected_flexmatch_score_table['main_interest_category'] = connected_flexmatch_score_table['interestcategory'].str.split('@').str[0]
# connected_flexmatch_score_table

In [75]:
# def normalize_influencer_scores(influencer_scale_names, influencer_scale_df_list, reverse_columns=None, feature_range=(0, 5)):

#     if reverse_columns is None:
#         reverse_columns = ['activity_score']

#     normalized_df_dict = {}

#     for name, df in zip(influencer_scale_names, influencer_scale_df_list):
#         cleaned = df.copy()

#         # 무한대 및 NaN 제거
#         float_cols = cleaned.select_dtypes(include='float64').columns
#         cleaned[float_cols] = cleaned[float_cols].replace([np.inf, -np.inf], np.nan)
#         cleaned = cleaned.dropna(subset=float_cols)

#         if cleaned.empty:
#             continue

#         norm_df = pd.DataFrame(index=cleaned.index)
#         for col in float_cols:

#             scaler = MinMaxScaler(feature_range=feature_range)
#             norm_col = scaler.fit_transform(cleaned[[col]])
#             if col in reverse_columns:
#                 norm_df[col] = feature_range[1] - norm_col.ravel()
#             else:
#                 norm_df[col] = norm_col.ravel()

#         # ID 및 이름, 스케일 타입 추가
#         norm_df['acnt_id'] = cleaned['acnt_id'].values
#         norm_df['acnt_nm'] = cleaned['acnt_nm'].values
#         norm_df['influencer_scale_type'] = name

#         normalized_df_dict[name] = norm_df

#     normalized_all_df = pd.concat(normalized_df_dict.values(), ignore_index=True)
#     normalized_all_dic = normalized_all_df.to_dict(orient='index')

#     return normalized_all_df, normalized_all_dic


In [ ]:
# def normalize_influencer_scores(
#     influencer_scale_names, 
#     influencer_scale_df_list, 
#     reverse_columns=None, 
#     log_columns=None, 
#     feature_range=(0, 5)  ## 해당 부분 1,5점으로 변경
# ):
#     if reverse_columns is None:
#         reverse_columns = []
#     if log_columns is None:
#         log_columns = []

#     normalized_df_dict = {}

#     for name, df in zip(influencer_scale_names, influencer_scale_df_list):
#         cleaned = df.copy()

#         # 무한대 및 NaN 제거
#         float_cols = cleaned.select_dtypes(include='float64').columns
#         cleaned[float_cols] = cleaned[float_cols].replace([np.inf, -np.inf], np.nan)
#         cleaned = cleaned.dropna(subset=float_cols)

#         if cleaned.empty:
#             continue

#         norm_df = pd.DataFrame(index=cleaned.index)
#         for col in float_cols:
#             col_data = cleaned[col]
#             if col in log_columns:
#                 col_data = np.log1p(col_data)
#             scaler = MinMaxScaler(feature_range=feature_range)
#             norm_col = scaler.fit_transform(col_data.values.reshape(-1, 1))
#             if col in reverse_columns:
#                 norm_df[col] = feature_range[1] - norm_col.ravel()
#             else:
#                 norm_df[col] = norm_col.ravel()

#         norm_df['acnt_id'] = cleaned['acnt_id'].values
#         norm_df['acnt_nm'] = cleaned['acnt_nm'].values
#         norm_df['influencer_scale_type'] = name

#         normalized_df_dict[name] = norm_df

#     normalized_all_df = pd.concat(normalized_df_dict.values(), ignore_index=True)
#     normalized_all_dic = normalized_all_df.to_dict(orient='index')

#     return normalized_all_df, normalized_all_dic


In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# def normalize_influencer_scores(
#     influencer_scale_names, 
#     influencer_scale_df_list, 
#     reverse_columns=None, 
#     log_columns=None, 
#     feature_range=(1, 5)
# ):
#     if reverse_columns is None:
#         reverse_columns = []
#     if log_columns is None:
#         log_columns = []

#     normalized_df_list = []

#     for name, df in zip(influencer_scale_names, influencer_scale_df_list):
#         cleaned = df.copy()

#         # 무한대 및 NaN 제거
#         float_cols = cleaned.select_dtypes(include='float64').columns
#         cleaned[float_cols] = cleaned[float_cols].replace([np.inf, -np.inf], np.nan)
#         cleaned = cleaned.dropna(subset=float_cols)

#         if cleaned.empty:
#             continue

#         # 관심 카테고리 첫 번째 값 추출
#         if 'interestcategory' in cleaned.columns:
#             cleaned['main_interest_category'] = cleaned['interestcategory'].apply(
#                 lambda x: str(x).split('@')[0] if pd.notnull(x) else '뷰티'
#             )
#         else:
#             cleaned['main_interest_category'] = '뷰티'

#         # 스케일 + 관심카테고리 조합별 정규화
#         grouped = cleaned.groupby('main_interest_category')

#         for category, group in grouped:
#             norm_df = pd.DataFrame(index=group.index)
#             for col in float_cols:
#                 col_data = group[col]
#                 if col in log_columns:
#                     col_data = np.log1p(col_data)
#                 scaler = MinMaxScaler(feature_range=feature_range)
#                 norm_col = scaler.fit_transform(col_data.values.reshape(-1, 1))
#                 if col in reverse_columns:
#                     norm_df[col] = feature_range[1] - norm_col.ravel()
#                 else:
#                     norm_df[col] = norm_col.ravel()

#             norm_df['acnt_id'] = group['acnt_id'].values
#             norm_df['acnt_nm'] = group['acnt_nm'].values
#             norm_df['influencer_scale_type'] = name
#             norm_df['main_interest_category'] = category

#             normalized_df_list.append(norm_df)

#     normalized_all_df = pd.concat(normalized_df_list, ignore_index=True)
#     normalized_all_dic = normalized_all_df.to_dict(orient='index')

#     return normalized_all_df, normalized_all_dic


In [104]:
nano = connected_flexmatch_score_table[connected_flexmatch_score_table['influencer_scale_type']=='nano']
micro = connected_flexmatch_score_table[connected_flexmatch_score_table['influencer_scale_type']=='micro']
mid = connected_flexmatch_score_table[connected_flexmatch_score_table['influencer_scale_type']=='mid']
macro = connected_flexmatch_score_table[connected_flexmatch_score_table['influencer_scale_type']=='macro']
mega = connected_flexmatch_score_table[connected_flexmatch_score_table['influencer_scale_type']=='mega']

In [118]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

def normalize_influencer_scores(
    influencer_scale_names, 
    influencer_scale_df_list, 
    reverse_columns=None, 
    log_columns=None, 
    feature_range=(1, 5)
):
    if reverse_columns is None:
        reverse_columns = []
    if log_columns is None:
        log_columns = []

    normalized_df_list = []

    for name, df in zip(influencer_scale_names, influencer_scale_df_list):
        cleaned = df.copy()

        # 무한대 및 NaN 제거
        float_cols = cleaned.select_dtypes(include='float64').columns
        cleaned[float_cols] = cleaned[float_cols].replace([np.inf, -np.inf], np.nan)
        cleaned = cleaned.dropna(subset=float_cols)

        if cleaned.empty:
            continue

        # 관심 카테고리 첫 번째 값 추출
        if 'interestcategory' in cleaned.columns:
            cleaned['main_interest_category'] = cleaned['interestcategory'].apply(
                lambda x: str(x).split('@')[0] if pd.notnull(x) else '뷰티'
            )
        else:
            cleaned['main_interest_category'] = '뷰티'

        # (관심카테고리, 스케일타입) 조합별 그룹화
        cleaned['influencer_scale_type'] = name  # 확실히 넣어두기
        grouped = cleaned.groupby(['main_interest_category', 'influencer_scale_type'])

        for (category, scale_type), group in grouped:
            norm_df = pd.DataFrame(index=group.index)

            # 그룹 내 인원 수 체크
            if len(group) == 1:
                # 1명인 경우 모든 점수를 1.0으로 고정
                for col in float_cols:
                    norm_df.loc[group.index, col] = 3.0
            else:
                # 여러명인 경우 정규화 진행
                for col in float_cols:
                    col_data = group[col]
                    if col in log_columns:
                        col_data = np.log1p(col_data)

                    max_val = col_data.max()
                    min_val = col_data.min()

                    if max_val == min_val:
                        # 모든 값이 동일하면 1.0 고정
                        norm_df.loc[group.index, col] = 3.0
                    else:
                        scaler = MinMaxScaler(feature_range=feature_range)
                        norm_col = scaler.fit_transform(col_data.values.reshape(-1, 1))
                        if col in reverse_columns:
                            norm_df[col] = feature_range[1] - norm_col.ravel()
                        else:
                            norm_df[col] = norm_col.ravel()

            # 공통 컬럼 복사
            norm_df['acnt_id'] = group['acnt_id'].values
            norm_df['acnt_nm'] = group['acnt_nm'].values
            norm_df['influencer_scale_type'] = scale_type
            norm_df['main_interest_category'] = category

            normalized_df_list.append(norm_df)

    normalized_all_df = pd.concat(normalized_df_list, ignore_index=True)
    normalized_all_dic = normalized_all_df.to_dict(orient='index')

    return normalized_all_df, normalized_all_dic


In [119]:
influencer_scale_names=['nano', 'micro', 'mid', 'macro', 'mega']
influencer_scale_df_list=[nano, micro, mid, macro, mega]

normalized_df, normalized_all_dic = normalize_influencer_scores(influencer_scale_names, influencer_scale_df_list, reverse_columns=None)


In [122]:
normalized_all_dic

{0: {'activity_score': 3.0,
  'follow_growth_rate': 3.0,
  'follower_retention_rate': 3.0,
  'avg_post_efficiency': 3.0,
  'avg_post_popularity': 3.0,
  'acnt_id': '17841450980480576',
  'acnt_nm': '__dongx2_',
  'influencer_scale_type': 'nano',
  'main_interest_category': '베이비/키즈'},
 1: {'activity_score': 1.4567394957983195,
  'follow_growth_rate': 1.0,
  'follower_retention_rate': 1.0,
  'avg_post_efficiency': 5.0,
  'avg_post_popularity': 5.0,
  'acnt_id': '17841400360358101',
  'acnt_nm': 'bro_bums',
  'influencer_scale_type': 'nano',
  'main_interest_category': '패션'},
 2: {'activity_score': 5.0,
  'follow_growth_rate': 5.0,
  'follower_retention_rate': 5.0,
  'avg_post_efficiency': 1.0,
  'avg_post_popularity': 3.0047435780677523,
  'acnt_id': '17841400624415491',
  'acnt_nm': 'itsme_ddonni',
  'influencer_scale_type': 'nano',
  'main_interest_category': '패션'},
 3: {'activity_score': 1.0,
  'follow_growth_rate': 5.0,
  'follower_retention_rate': 5.0,
  'avg_post_efficiency': 3.453

In [123]:
ssh.insert_query_with_lookup('op_mem_seller_score', list(normalized_all_dic.values()))

INSERT INTO op_mem_seller_score (activity_score, follow_growth_rate, follower_retention_rate, avg_post_efficiency, avg_post_popularity, acnt_id, acnt_nm, influencer_scale_type, main_interest_category, member_uid, user_id, is_connected) VALUES (%(activity_score)s, %(follow_growth_rate)s, %(follower_retention_rate)s, %(avg_post_efficiency)s, %(avg_post_popularity)s, %(acnt_id)s, %(acnt_nm)s, %(influencer_scale_type)s, %(main_interest_category)s, %(member_uid)s, %(user_id)s, %(is_connected)s)
inserted acnt_id: 17841450980480576
INSERT INTO op_mem_seller_score (activity_score, follow_growth_rate, follower_retention_rate, avg_post_efficiency, avg_post_popularity, acnt_id, acnt_nm, influencer_scale_type, main_interest_category, member_uid, user_id, is_connected) VALUES (%(activity_score)s, %(follow_growth_rate)s, %(follower_retention_rate)s, %(avg_post_efficiency)s, %(avg_post_popularity)s, %(acnt_id)s, %(acnt_nm)s, %(influencer_scale_type)s, %(main_interest_category)s, %(member_uid)s, %(use

In [80]:
# def normalize_influencer_scores(influencer_scale_names, influencer_scale_df_list, reverse_columns=None, feature_range=(0, 5)):

#     if reverse_columns is None:
#         reverse_columns = ['activity_score']

#     normalized_df_list = []

#     for name, df in zip(influencer_scale_names, influencer_scale_df_list):
#         cleaned = df.copy()

#         if 'interest_category' not in cleaned.columns:
#             raise ValueError(f"'{name}' 데이터프레임에 'interest_category' 컬럼이 없습니다.")

#         # 무한대 및 NaN 제거
#         float_cols = cleaned.select_dtypes(include='float64').columns
#         cleaned[float_cols] = cleaned[float_cols].replace([np.inf, -np.inf], np.nan)
#         cleaned = cleaned.dropna(subset=float_cols)

#         if cleaned.empty:
#             continue

#         # 관심 카테고리별로 그룹화하여 정규화
#         grouped = cleaned.groupby('interest_category')

#         for category, group_df in grouped:
#             if group_df.empty:
#                 continue

#             norm_df = pd.DataFrame(index=group_df.index)

#             for col in float_cols:
#                 scaler = MinMaxScaler(feature_range=feature_range)
#                 norm_col = scaler.fit_transform(group_df[[col]])

#                 if col in reverse_columns:
#                     norm_df[col] = feature_range[1] - norm_col.ravel()
#                 else:
#                     norm_df[col] = norm_col.ravel()

#             norm_df['acnt_id'] = group_df['acnt_id'].values
#             norm_df['acnt_nm'] = group_df['acnt_nm'].values
#             norm_df['interest_category'] = group_df['interest_category'].values
#             norm_df['influencer_scale_type'] = name

#             normalized_df_list.append(norm_df)

#     normalized_all_df = pd.concat(normalized_df_list, ignore_index=True)
#     normalized_all_dic = normalized_all_df.to_dict(orient='index')

#     return normalized_all_df, normalized_all_dic


##### 크리에이터 광고 효율성 

In [81]:
revenue_dic = {
    'acnt_nm' : ['s_h_j_', 'siwolbubu_hyun', 'bong_camper83', 'binwoos', 'seojinii_', 'tingkerhee'],
    'sell_type' : ['flexmatch', 'other', 'flexmatch', 'flexmatch', 'other', 'other'],
    'total_revenue' : [6906000, 10937105, 7233100, 8759000, 7939664, 38449720],
    'total_order_cnt' : [84, 132, 122, 88, 89, 471]
    }

revenue_df = pd.DataFrame(revenue_dic)
revenue_merged_df = pd.merge(media_engagement_merged_df, revenue_df, on='acnt_nm', how='left')
revenue_merged_df = revenue_merged_df[['acnt_id', 'acnt_nm', 'follower_cnt', 'follow_cnt', 'media_cnt', 'sell_type', 'total_revenue', 'total_order_cnt', 'media_id', 'like_cnt', 'cmnt_cnt']]
revenue_merged_df

revenue_merged_df['post_efficiency'] = ((revenue_merged_df['like_cnt'] + revenue_merged_df['cmnt_cnt']) / revenue_merged_df['follower_cnt']) * 100
revenue_df_total = revenue_merged_df.groupby(['acnt_id', 'acnt_nm']).agg({
    'post_efficiency' : 'mean',
    'total_order_cnt' : 'first'
}).dropna()

revenue_df_total['advertisement_efficiency'] = (revenue_df_total['total_order_cnt'] / revenue_df_total['post_efficiency']) 
revenue_df_total.sort_values(by='advertisement_efficiency', ascending=False)

revenue_merged_df['engagement_per_post'] = ((revenue_merged_df['like_cnt'] + revenue_merged_df['cmnt_cnt']) / 25)
revenue_df_total = revenue_merged_df.groupby(['acnt_id', 'acnt_nm']).agg({
    'engagement_per_post' : 'mean',
    'total_revenue' : 'first',
    'total_order_cnt' : 'first'
}).dropna()

# 팔로워를 생각하지 않고, 25개의 콘텐츠 단위당 반응 비율을 계산
revenue_df_total['advertisement_efficiency'] = (revenue_df_total['total_order_cnt'] / revenue_df_total['engagement_per_post'] * 25) * 100
revenue_df_total.sort_values(by='advertisement_efficiency', ascending=False)

KeyError: "['follower_cnt', 'media_cnt'] not in index"

콘텐츠 효율성이 아닌 팔로워 참여도를 기준으로 계산했을 때

In [ ]:
engaged_df_2 = engaged_df.copy()
revenue_merged_df_2 = revenue_merged_df.groupby(['acnt_id', 'acnt_nm'])[['total_revenue', 'total_order_cnt']].first().dropna().reset_index()
revenue_df_total_2 = pd.merge(engaged_df_2, revenue_merged_df_2, on='acnt_id')

revenue_df_total_2['advertisement_efficiency'] = revenue_df_total_2['total_order_cnt'] / revenue_df_total_2['follower_total_engagement']
revenue_df_total_2[['acnt_id', 'acnt_nm', 'total_revenue', 'total_order_cnt', 'follower_total_engagement', 'advertisement_efficiency']].sort_values(by='advertisement_efficiency', ascending=False)

table merged

In [ ]:
# 크리에이터 활동성
creator_activity_score = activity_df[['acnt_id', 'avg_upload_interval']]

In [ ]:
# 트렌드지수
creator_trend_score = time_series_merged_df[['acnt_id', 'trend_score']]

In [ ]:
# 팔로워 참여도
follower_engagement = engaged_df[['acnt_id', 'follower_total_engagement']]

In [ ]:
# 팔로워 충성도
follower_loyalty = time_series_merged_df_copy[['acnt_id', 'follower_retention_rate']]

In [ ]:
# 콘텐츠 효율성
post_efficiency = post_efficiency_df.copy()

In [ ]:
# 광고효율성
revenue_df_total = revenue_df_total.reset_index()
advertisement_efficiency = revenue_df_total[['acnt_id', 'advertisement_efficiency']]

In [ ]:
df_list = [creator_activity_score, creator_trend_score, follower_engagement, follower_loyalty, post_efficiency, advertisement_efficiency]

In [ ]:
from functools import reduce

flexmatch_score = reduce(lambda left, right: pd.merge(left, right, on='acnt_id', how='left'), df_list)

In [ ]:
user_info_nm = user_info_2[['acnt_id', 'acnt_nm']]
flexmatch_score = pd.merge(flexmatch_score, user_info_nm, on='acnt_id')

In [ ]:
flexmatch_score = flexmatch_score[['acnt_id', 'acnt_nm', 'avg_upload_interval', 'trend_score', 'follower_total_engagement', 'follower_retention_rate', 'avg_post_efficiency', 'advertisement_efficiency']]
flexmatch_score.rename(columns={
    'avg_upload_interval' : '크리에이터 활동성',
    'trend_score' : '트렌드지수(팔로워순증가량)', 
    'follower_total_engagement' : '콘텐츠 참여도' ,
    'follower_retention_rate' : '팔로워 충성도',
    'avg_post_efficiency' : '콘텐츠 효율성',
    'advertisement_efficiency' : '광고 효율성'
}, inplace=True)

In [ ]:
flexmatch_score_2 = flexmatch_score.copy()
flexmatch_score_2.dropna(inplace=True)

In [ ]:
flexmatch_score_2

,acnt_id,acnt_nm,크리에이터 활동성,트렌드지수(팔로워순증가량),콘텐츠 참여도,팔로워 충성도,콘텐츠 효율성,광고 효율성
0,17841400361359004,s_h_j_,1.166667,0.057866,110.188904,100.00,0.045178,1859.316062
1,17841400561503844,binwoos,0.500000,-0.014956,626.642600,99.99,0.169730,518.470975
2,17841400591698216,tingkerhee,2.416667,-0.018509,2065.579146,99.98,0.483403,974.343202
4,17841401506106699,siwolbubu_hyun,0.958333,-0.075045,2094.996055,99.92,0.556736,237.096389
6,17841402936102997,seojinii_,2.875000,-0.002312,166.809026,100.00,0.240359,370.279723
10,17841453615191128,bong_camper83,1.500000,0.281793,2526.578680,100.00,2.506526,48.672935


In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# columns = flexmatch_score_2.select_dtypes(include='float64').columns
# minmax = MinMaxScaler(feature_range=(0, 5))
# flexmatch_score_norm = minmax.fit_transform(flexmatch_score_2[columns])

# flexmatch_score_norm_df = pd.DataFrame(flexmatch_score_norm, columns=columns, index=flexmatch_score_2.index)
# flexmatch_score_norm_df['acnt_id'] = flexmatch_score_2['acnt_id']
# flexmatch_score_norm_df['acnt_nm'] = flexmatch_score_2['acnt_nm']

# flexmatch_score_norm_df['avg_upload_interval'] = 5 - flexmatch_score_norm_df['avg_upload_interval']
# flexmatch_score_norm_df


In [ ]:
from sklearn.preprocessing import MinMaxScaler

columns = flexmatch_score_2.select_dtypes(include='float64').columns
minmax = MinMaxScaler(feature_range=(0, 5))
flexmatch_score_norm = minmax.fit_transform(flexmatch_score_2[columns])

flexmatch_score_norm_df = pd.DataFrame(flexmatch_score_norm, columns=columns, index=flexmatch_score_2.index)
flexmatch_score_norm_df['acnt_id'] = flexmatch_score_2['acnt_id']
flexmatch_score_norm_df['acnt_nm'] = flexmatch_score_2['acnt_nm']

flexmatch_score_norm_df['크리에이터 활동성'] = 5 - flexmatch_score_norm_df['크리에이터 활동성']
flexmatch_score_norm_df

# 지금 이거 정규화 할 때는 그냥 이사람들을 전부 다 합쳐서 함께 정규화를 진행해서 그런데, 원래라면은 팔로워를 기준으로 뭐 메가 인플루언서, 마이크로 인플루언서 이런식으로 기준을 나눠서 정규화를 해야 조금 더 정확한 값이 될 것 같음.


,크리에이터 활동성,트렌드지수(팔로워순증가량),콘텐츠 참여도,팔로워 충성도,콘텐츠 효율성,광고 효율성,acnt_id,acnt_nm
0,3.596491,1.862339,0.000000,5.000,0.000000,5.000000,17841400361359004,s_h_j_
1,5.000000,0.841960,1.068647,4.375,0.253016,1.297324,17841400561503844,binwoos
2,0.964912,0.792182,4.046099,3.750,0.890213,2.556192,17841400591698216,tingkerhee
4,4.035088,0.000000,4.106968,0.000,1.039182,0.520322,17841401506106699,siwolbubu_hyun
6,0.000000,1.019133,0.117159,5.000,0.396492,0.888101,17841402936102997,seojinii_
10,2.894737,5.000000,5.000000,5.000,5.000000,0.000000,17841453615191128,bong_camper83


In [ ]:
# flexmatch_score_norm_df.to_csv("flexmatch_score_test.csv")